In [1]:
import numpy as np
import pickle
import random
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader,Dataset,SequentialSampler,BatchSampler


In [2]:
fft_class_train='./samples/fft_class_train.pkl'
fft_class_test= './samples/fft_class_test.pkl'

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import pickle

class CustomDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'rb') as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features, label = self.data[idx]
        features = torch.FloatTensor(features)
        label = torch.tensor(label, dtype=torch.float32)
        return features, label


# Create train and test datasets
train_dataset = CustomDataset(fft_class_train)
test_dataset = CustomDataset(fft_class_test)

# Create train and test loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000025B1F52B490>>
Traceback (most recent call last):
  File "c:\Users\26413\.conda\envs\py1\Lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
# Assuming you have a DataLoader named train_loader
# Access the first batch of data
inputs, _ = next(iter(train_loader))

# Check the shape of the input tensor
print("Input shape:", inputs.shape)


Input shape: torch.Size([32, 1000])


In [ ]:
# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid activation for binary classification
        return x
# Assuming 'inputs' is your training data
# Define the input size
input_size = inputs.shape[1]

# Initialize the model
model = NeuralNetwork(input_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Train the model
model.train()
epochs = 70
losses = []
for epoch in range(epochs):
    for inputs, outputs in train_loader:
        y_pred = model.forward(inputs)
        # Remove the singleton dimension from y_pred
        y_pred = torch.squeeze(y_pred, dim=1)
        loss = criterion(y_pred, outputs.float())  # Ensure outputs are float
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch {}/{}: Loss: {:.4f}'.format(epoch + 1, epochs, loss.item()))


Epoch 1/70: Loss: 0.6843
Epoch 2/70: Loss: 0.6679
Epoch 3/70: Loss: 0.5275
Epoch 4/70: Loss: 0.7170
Epoch 5/70: Loss: 0.6974
Epoch 6/70: Loss: 0.5431
Epoch 7/70: Loss: 0.6038
Epoch 8/70: Loss: 0.6014
Epoch 9/70: Loss: 0.6936
Epoch 10/70: Loss: 0.6216
Epoch 11/70: Loss: 0.4751
Epoch 12/70: Loss: 0.4884
Epoch 13/70: Loss: 0.6580
Epoch 14/70: Loss: 0.4266
Epoch 15/70: Loss: 0.4151
Epoch 16/70: Loss: 0.4402
Epoch 17/70: Loss: 0.5426
Epoch 18/70: Loss: 0.5948
Epoch 19/70: Loss: 0.4927
Epoch 20/70: Loss: 0.4447
Epoch 21/70: Loss: 0.6239
Epoch 22/70: Loss: 0.6506
Epoch 23/70: Loss: 0.4862
Epoch 24/70: Loss: 0.6300
Epoch 25/70: Loss: 0.6073
Epoch 26/70: Loss: 0.4930
Epoch 27/70: Loss: 0.4913
Epoch 28/70: Loss: 0.4912
Epoch 29/70: Loss: 0.5203
Epoch 30/70: Loss: 0.5135
Epoch 31/70: Loss: 0.5947
Epoch 32/70: Loss: 0.6090
Epoch 33/70: Loss: 0.5299
Epoch 34/70: Loss: 0.7509
Epoch 35/70: Loss: 0.5685
Epoch 36/70: Loss: 0.5732
Epoch 37/70: Loss: 0.5551
Epoch 38/70: Loss: 0.4423
Epoch 39/70: Loss: 0.

In [ ]:
# Switch to evaluation mode
model.eval()

total_accuracy = 0
total_loss = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        # Perform forward pass
        test_preds = model(inputs)
        
        # Remove the singleton dimension from the model's output
        test_preds = torch.squeeze(test_preds, dim=1)
        
        # Calculate loss
        test_loss = criterion(test_preds, targets.float())  # Ensure targets are float
        total_loss += test_loss.item()
        
        # Apply sigmoid activation to get predicted probabilities
        predicted = (test_preds >= 0.5).float()  # Binary classification
        
        # Calculate accuracy
        accuracy = (predicted == targets).float().mean().item()
        total_accuracy += accuracy

# Calculate average loss
avg_loss = total_loss / len(test_loader)

# Calculate average accuracy
avg_accuracy = total_accuracy / len(test_loader)

print(f'Test Loss: {avg_loss}, Average Accuracy: {avg_accuracy}')


Test Loss: 1.856090553826846, Average Accuracy: 0.49121231401956167
